# MusicBrainz editor statistics extraction
This notebook generates statistical data from the editor table of a MusicBrainz database.

In [4]:
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import numpy as np
import pickle

/home/user/.venvs/dataviz/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
connection = pg.connect("host=localhost dbname=musicbrainz_db user=musicbrainz password=musicbrainz")

In [3]:
df = psql.read_sql_query("SELECT * FROM editor", connection)

In [4]:
def in_qrange(s, q):
    return s.between(*s.quantile(q=q))

# Filtering quantils prevents a couple extremes from causing dozens of empty bins during binning
def filter_quantiles(s, q):
    return s.loc[s.transform(in_qrange, q=q)]

def bin(s, bin_count):
    return pd.cut(s, bin_count)

def hist(s, head=None):
    hist = s.value_counts(dropna=False)
    
    if head is not None:
        return hist.head(head)
    else:
        return hist

def bin_hist(s, bin_count):
    return hist(bin(s, bin_count))
    
def filter_bin_hist(s, bin_count, q):
    return bin_hist(filter_quantiles(s, q), bin_count)

In [5]:
priv_hist = hist(df["privs"])
area_hist = hist(df["area"], 100)
gender_hist = hist(df["gender"])

In [6]:
emaildomain_hist_raw = hist(df["email"].str.replace(r".*@", ""), 100)
# As email domains are personally identifying data, drop domains with fewer than ten occurances
emaildomain_hist = emaildomain_hist_raw.where(lambda x : x >= 10).dropna()

In [7]:
login_diffs = pd.Series([(row["last_login_date"] - row["member_since"])/pd.Timedelta(hours=1) for index, row in df.iterrows()])
login_diff_hist = filter_bin_hist(login_diffs, 100, [0, 0.99])

In [8]:
confirmation_diffs = pd.Series([(row["email_confirm_date"] - row["member_since"])/pd.Timedelta(hours=1) for index, row in df.iterrows()])
confirmation_diff_hist = filter_bin_hist(confirmation_diffs, 100, [0, 0.99])

In [9]:
update_diffs = pd.Series([(row["last_updated"] - row["member_since"])/pd.Timedelta(hours=1) for index, row in df.iterrows()])
update_diff_hist = filter_bin_hist(update_diffs, 100, [0, 0.95])

In [10]:
birth_date_hist = hist(df["birth_date"], 100)
# I considered splitting this into year/month/day but with so few entries, a set/not-set flag should probably work well enough

In [11]:
output = [priv_hist, area_hist, gender_hist, emaildomain_hist, login_diff_hist, confirmation_diff_hist, update_diff_hist, birth_date_hist]

In [1]:
with open("stats.pickle", "wb") as f:
    pickle.dump(output, f, protocol=pickle.HIGHEST_PROTOCOL)

NameError: name 'pickle' is not defined

In [16]:
np.logspace(np.log10(1),np.log10(1000), 100)

array([   1.        ,    1.07226722,    1.149757  ,    1.23284674,
          1.32194115,    1.41747416,    1.51991108,    1.62975083,
          1.7475284 ,    1.87381742,    2.009233  ,    2.15443469,
          2.3101297 ,    2.47707636,    2.65608778,    2.84803587,
          3.05385551,    3.27454916,    3.51119173,    3.76493581,
          4.03701726,    4.32876128,    4.64158883,    4.97702356,
          5.33669923,    5.72236766,    6.13590727,    6.57933225,
          7.05480231,    7.56463328,    8.11130831,    8.69749003,
          9.32603347,   10.        ,   10.72267222,   11.49756995,
         12.32846739,   13.21941148,   14.17474163,   15.19911083,
         16.29750835,   17.475284  ,   18.73817423,   20.09233003,
         21.5443469 ,   23.101297  ,   24.77076356,   26.56087783,
         28.48035868,   30.53855509,   32.74549163,   35.11191734,
         37.64935807,   40.37017259,   43.28761281,   46.41588834,
         49.77023564,   53.36699231,   57.22367659,   61.35907

In [17]:
np.logspace(np.log10(1),np.log10(20000), 100)

array([1.00000000e+00, 1.10520985e+00, 1.22148882e+00, 1.35000147e+00,
       1.49203493e+00, 1.64901170e+00, 1.82250398e+00, 2.01424935e+00,
       2.22616822e+00, 2.46038305e+00, 2.71923959e+00, 3.00533038e+00,
       3.32152074e+00, 3.67097745e+00, 4.05720044e+00, 4.48405790e+00,
       4.95582496e+00, 5.47722657e+00, 6.05348477e+00, 6.69037100e+00,
       7.39426394e+00, 8.17221335e+00, 9.03201070e+00, 9.98226721e+00,
       1.10325001e+01, 1.21932277e+01, 1.34760754e+01, 1.48938913e+01,
       1.64608754e+01, 1.81927217e+01, 2.01067752e+01, 2.22222061e+01,
       2.45602011e+01, 2.71441762e+01, 3.00000109e+01, 3.31563076e+01,
       3.66446778e+01, 4.05000589e+01, 4.47610641e+01, 4.94703690e+01,
       5.46751392e+01, 6.04275025e+01, 6.67850710e+01, 7.38115184e+01,
       8.15772173e+01, 9.01599442e+01, 9.96456586e+01, 1.10129364e+02,
       1.21716057e+02, 1.34521786e+02, 1.48674803e+02, 1.64316857e+02,
       1.81604609e+02, 2.00711203e+02, 2.21827999e+02, 2.45166490e+02,
      